In [1]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
import warnings
import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

d:\GitHub repos\ADL\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
randomState = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])

#* 90/10 split for training and final test
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=randomState, stratify=Y)

Using device: cuda


In [48]:
# display(raw_dataset.head())
display
Norm_function = MinMaxScaler() # Normalization function
test_norm = Norm_function.fit_transform(X_FOR_FOLDS)
display(test_norm[:5])
test_norm = pd.DataFrame(test_norm)
community_cols = [col for col in test_norm.columns if col.startswith('Community')] 
print("Community columns:", community_cols)

array([[4.82758621e-01, 1.00000000e+00, 4.41355996e-03, 3.52019713e-04,
        5.50173480e-05, 1.54148472e-01, 1.16646416e-02, 1.81195280e-01,
        4.56057007e-01, 2.63565891e-01, 4.79452055e-02, 2.69058296e-01,
        1.61538462e-01, 2.77777778e-01, 7.54716981e-01, 5.11363636e-02,
        3.79865031e-02, 6.96048632e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.89655172e-01, 0.00000000e+00, 2.03775002e-02, 5.05192291e-01,
        1.95487173e-04, 1.44104803e-01, 5.46780073e-03, 3.52874001e-01,
        4.70308789e-01, 2.94573643e-01, 6.16438356e-02, 1.92825112e-01,
        1.00000000e-01, 3.51851852e-01, 8.11320755e-01, 5.49242424e-02,
        3.38270871e-02, 9.72644377e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000

AttributeError: 'int' object has no attribute 'startswith'

In [21]:
binary_cols = [col for col in test_norm if set(test_norm[col].unique()).issubset({0.0, 1.0})]
print("Binary columns: ", binary_cols)  

Binary columns:  [1, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


In [106]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, random_state=None, oversampler=None):
    """
    Generates stratified folds with specified normalization and oversampling.
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []
    
    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        
        # IsolationForest for outlier removal (optional)
        iso_forest = IsolationForest(contamination=0.05, random_state=random_state)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            outliers = iso_forest.fit_predict(X_train)    
        X_train = X_train[outliers == 1]
        Y_train = Y_train[outliers == 1]
        
        # Scale the entire data (binary and continuous together)
        X_train_scaled = normalisation_method.fit_transform(X_train)
        X_test_scaled = normalisation_method.transform(X_test)

        # Handle oversampling if needed
        if oversampler:
            # Apply oversampling to both features and target
            X_train_scaled, Y_train = oversampler.fit_resample(X_train_scaled, Y_train)

        # Convert scaled data back to DataFrame with the correct column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
        
        # Handle community columns
        community_cols = [col for col in X_train_scaled.columns if col.startswith('Community')] 

        # Check for rows where multiple communities are flagged
        for idx, row in X_train_scaled[community_cols].iterrows():
            if set(np.unique(row)) != {0, 1}:  # If the unique values aren't just 0 or 1
                # Fix row by ensuring only one community is marked
                X_train_scaled.loc[idx, community_cols] = 0  # Set all community columns to 0
                max_col = row.idxmax()  # Find the column with the maximum value
                X_train_scaled.at[idx, max_col] = 1  # Set the column with the max value to 1

        # Ensure 'gender' is still binary (0 or 1)
        if 'Gender' in X_train_scaled.columns:
            X_train_scaled['Gender'] = (X_train_scaled['Gender'] > 0.5).astype(int)
            X_test_scaled['Gender'] = (X_test_scaled['Gender'] > 0.5).astype(int)
        
        # Append the processed fold to the list
        kFolds_list.append((X_train_scaled, X_test_scaled, Y_train, Y_test))
        
        print(f"Fold: {fold+1}, Train: {kFolds_list[fold][0].shape}, Test: {kFolds_list[fold][1].shape}")
    
    return kFolds_list


In [107]:
oversampler = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=5)
normalisation_method = MinMaxScaler() # Normalization function

Folds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 

                         oversampler = oversampler, random_state=42)

Fold: 1, Train: (7981, 28), Test: (1149, 28)
Fold: 2, Train: (7859, 28), Test: (1149, 28)
Fold: 3, Train: (7906, 28), Test: (1148, 28)
Fold: 4, Train: (7942, 28), Test: (1148, 28)
Fold: 5, Train: (7958, 28), Test: (1148, 28)


In [108]:
Folds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 

                         oversampler = None, random_state=42)

Fold: 1, Train: (4363, 28), Test: (1149, 28)
Fold: 2, Train: (4363, 28), Test: (1149, 28)
Fold: 3, Train: (4364, 28), Test: (1148, 28)
Fold: 4, Train: (4364, 28), Test: (1148, 28)
Fold: 5, Train: (4364, 28), Test: (1148, 28)
